In [1]:
import enum
import gym
from gym import error, spaces, utils
from gym.utils import seeding
from gym.spaces import Discrete, Tuple
import pickle
import pandas as pd
from random import randint, getrandbits
import datetime
from datetime import timedelta
import os
from datetime import date
import copy
from timeit import default_timer as timer

pd.options.display.max_columns = None

import math, random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

# from solver import *
from vrp import *

### Sketch of algorithm

- data loading
- for a given date
  - append arrival cars to plants
  - determine next destinations
  - do vdc decision:
    - for each period
      - for each link
        - get state
        - send any full trucks/rails -> update dataframe
        - determine action on the link
        - process action
  - determine next destinations and delivery      
  - do delivery:
    - for each VDC
      - solve delivery problem
      - update dataframe


In [2]:



class DQN(nn.Module):
    def __init__(self, num_inputs, num_actions):
        super(DQN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(num_inputs, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, num_actions*2)
        )
        self.num_actions = num_actions
        
    def forward(self, x):
        return self.layers(x)

class INFORMSsolver:
    def __init__(self, distance_matrix, location_index_dic, location, expdir='../../Exp', year='2016', route_type=1, vehicle_type=1, restore_snapshot=None):


        # route_type
        # 1: static 
        # 2: dynamic

        # vehicle_type
        # 1: non-AV
        # 2: AV


        current_date = pd.to_datetime(f"{year}-01-01")

        
        
        datadir = f'{expdir}/{year}'
        commondir = f'{expdir}'

#         distance_matrix = pickle.load(open(f'{commondir}/dist_mat_9361.dump', 'rb'))
#         location_index_dic = pickle.load(open(f'{commondir}/location_index_9361.dump', 'rb'))        
#         location = pd.read_excel(f'{commondir}/Input_Cost%2C+Location.xlsx')


        shipment_master_df = pickle.load(open(f'{datadir}/shipment.dump', 'rb'))
        vdc_info_dic = pickle.load(open(f'{datadir}/vdc_info.dump', 'rb'))

        if route_type == 1:
            route_dic = pickle.load(open(f'{datadir}/route_static.dump', 'rb'))
        else:
            route_dic = pickle.load(open(f'{datadir}/route_dynamic2.dump', 'rb'))


        VDC = vdc_info_dic.keys()



        Links = [(i,j) for i in VDC for j in VDC if i != j]

        Plants = ('3A','FF','RO','SO')


        # maximum number of days a car can stay at a VDC
        MAX_STAY_DAYS = 7

        # min dwell time (days)
        MIN_DWELL_TIME = 2

        # decision interval (hours)
        DECISION_INTERVAL = 6

        # time spent at a VDC before delivery (days)
        DELIVERY_TIME = 2

        # how many future time periods the state considers
        NUM_LOOKING_FORWARD = 8


        # General information of each link
        # Link_info['3A']['SO'] = {'distance':..., 'time':..., 'rail':...}
        Link_info = {
            i: {
                j: {
                    'distance': distance_matrix[location_index_dic[i]][location_index_dic[j]],
                    'rail': vdc_info_dic[i]['rail'] and vdc_info_dic[j]['rail']
                }
                for j in VDC
                if i != j
            }
            for i in VDC
        }

        for i in VDC:
            for j in VDC:
                if i != j:
                    Link_info[i][j]['time'] = Link_info[i][j]['distance'] / (10.0 if Link_info[i][j]['rail'] else 30.0)

        # cars at each vdc
        VDC_all_df = {
            v: pd.DataFrame(columns=pd.Index(['vin', 'vdc', 'arrival_time', 'dealer', 'plant', 'pd_time', 'route', 'arcs', 'next', 'delivery'], dtype='object'))
            for v in VDC
        }
        
        
        # VRP solver
        self.vrp = VRP(list(VDC))
        self.vrp.location = location
        self.vrp.dist = distance_matrix
        self.vrp.ind = location_index_dic
        
#         self.vrp = VRP(location, distance_matrix, location_index_dic, list(VDC))
        

        model = DQN(NUM_LOOKING_FORWARD * 2, 1)
        model.load_state_dict(torch.load(f'{commondir}/INTERVER6_model_static_dic_256_10000_100.pt'))
        model.eval()
        
        self.model = model

        # problem data
        self.year = year
        self.current_date = current_date
        self.expdir = expdir
        self.datadir = datadir
        self.commondir = commondir
        self.distance_matrix = distance_matrix
        self.location_index_dic = location_index_dic
        self.shipment_master_df = shipment_master_df
        self.vdc_info_dic = vdc_info_dic
        self.route_dic = route_dic
        self.route_type = route_type
        self.vehicle_type = vehicle_type
        self.VDC = VDC
        self.Links = Links
        self.Plants = Plants
        self.Link_info = Link_info
        self.VDC_all_df = VDC_all_df
        


        # parameters
        self.MAX_STAY_DAYS = MAX_STAY_DAYS
        self.MIN_DWELL_TIME = MIN_DWELL_TIME
        self.DECISION_INTERVAL = DECISION_INTERVAL
        self.DELIVERY_TIME = DELIVERY_TIME
        self.NUM_LOOKING_FORWARD = NUM_LOOKING_FORWARD
        self.VRP_PERIODS = 1
        self.VRP_TIMEOUT = 10
        
        
        # results
        self.reset_results_df()
#         self.result_vdc_to_vdc_df = pd.DataFrame(columns=pd.Index(['depart_vdc', 'depart_time', 'arrival_vdc', 'arrival_time', 'method', 'vins', 'num_vins'], dtype='object'))
#         self.result_delivery_df = pd.DataFrame(columns=pd.Index(['vdc', 'shipment_id', 'depart_time', 'path', 'vins', 'cost', 'method'], dtype='object'))
#         self.result_route_details_df = pd.DataFrame(columns=pd.Index(['vin', 'loc', 'arrive_time', 'depart_time', 'depart_mode', 'shipment_id', 'shipment_cost'], dtype='object'))


        # restore status from snapshot
        if restore_snapshot is not None:
            snapshot = pickle.load(open(restore_snapshot, 'rb'))
            self.current_date = snapshot['date']
            self.VDC_all_df = {**self.VDC_all_df, **snapshot['vdc_status']}
            self.result_vdc_to_vdc_df = snapshot['vdc_to_vdc']
            self.result_delivery_df = snapshot['delivery']
            if 'depart_time' not in self.result_delivery_df:
                self.result_delivery_df['depart_time'] = [p[0][1] for p in self.result_delivery_df.path]
            

        

    def append_cars_arrive_today(self):
        num_cars = 0
        for p in self.Plants:
            arrive_today_df = self.shipment_master_df[
                (self.shipment_master_df.Plant.values == p) &
                (self.shipment_master_df.Plant_Arrival_Time >= self.current_date) &
                (self.shipment_master_df.Plant_Arrival_Time < self.current_date + timedelta(days=1))
            ]
            for index, row in arrive_today_df.iterrows():
                routes = self.route_dic[(row.Plant, row.Dealer)]['path'][:3]
                self.VDC_all_df[p].loc[row.VIN] = [
                    row.VIN, 
                    row.Plant, 
                    row.Plant_Arrival_Time, 
                    str(row.Dealer), 
                    row.Plant, 
                    row.Plant_Arrival_Time, 
                    routes, 
                    set([a for route in routes for a in route]),
                    None, 
                    row.Plant not in self.VDC
                ]
            num_cars += len(arrive_today_df)
        return num_cars
                
    def determine_next_dest(self):
        if self.route_type == 1: # static routing

            def next_dest_in_route(r,v):
                for a in r:
                    if a[0] == v:
                        return a[1]
                else:
                    return None

            for vdc in self.VDC:
                next_dest = [
                    str(next_dest_in_route(row.route[0], vdc))
                    for vin, row in self.VDC_all_df[vdc].iterrows()
                ]

                self.VDC_all_df[vdc].next = next_dest
                
        elif self.route_type == 2: # dynamic routing


            def next_dest_in_route(routes,v):
                for r in routes:
                    for a in r:
                        if a[0] == v:
                            return a[1]
                else:
                    qqq = 1/0
                    return None
                
            # first use static routes
            for vdc in self.VDC:
                next_dest = [
                    str(next_dest_in_route(row.route, vdc))
                    for vin, row in self.VDC_all_df[vdc].iterrows()
                ]

                self.VDC_all_df[vdc].next = next_dest
                

            
            num_changes = 0
            # aggregate cars
            for vdc in self.VDC:

                cars_to_determine_df = self.VDC_all_df[vdc]

                cars_can_send_df = cars_to_determine_df[(cars_to_determine_df.arrival_time < (self.current_date-timedelta(days=self.MIN_DWELL_TIME))) & (cars_to_determine_df.next.isin(self.VDC))]
                num_cars_dic = cars_can_send_df.next.value_counts().sort_values().to_dict()

                
                for next_dest, num_cars in num_cars_dic.items():
                    if num_cars < 3:
#                         print(f'Small cars: VDC:{vdc}, {num_cars_dic}')
                        small_num_cars = cars_can_send_df[cars_can_send_df.next==next_dest].index
                    #     small_num_cars = cars_can_send_df.iloc[:5].index

                        for vin in small_num_cars:
                            # possible alternative destinations
                            alt_next_dest = [
                                a[1] 
                                for a in cars_to_determine_df.loc[vin].arcs
                                if a[0] == vdc and a[1] in self.VDC and a[1] != cars_to_determine_df.loc[vin].next
                            ]
                            if len(alt_next_dest) > 1:
                                # num of cars of each alternative
                                num_alt_dests = [num_cars_dic[v] if v in num_cars_dic else 0 for v in alt_next_dest]

                                # the largest num of cars
                                max_num_alt_dests = max(num_alt_dests)

                                # the alternative is better
                                if max_num_alt_dests > num_cars_dic[cars_to_determine_df.loc[vin].next]:
                                    # change the next destination
                                    self.VDC_all_df[vdc].loc[vin, 'next'] = alt_next_dest[num_alt_dests.index(max(num_alt_dests))]
                                    num_changes += 1
#                                     print(f'Dest change: VDC:{vdc}, VIN:{next_dest} -> {alt_next_dest[num_alt_dests.index(max(num_alt_dests))]}')

            print(f'\t{num_changes} route changes!')
#             for v in self.VDC:
#                 self.determine_dynamic_routing(v)
            
        
        # determine delivery
        for v in self.VDC:
            self.VDC_all_df[v].delivery = ~self.VDC_all_df[v].next.isin(self.VDC)
            
            
#     def determine_dynamic_routing(self, vdc):
        
        
#         # def dertermine_dynamic_routing(self, vdc):
#         cars_to_determine_df = self.VDC_all_df[vdc]

#         NC_dic = {
#             vin: [a[1] for a in arcs if a[0]==vdc]
#             for (vin,arcs) in zip(cars_to_determine_df.index, cars_to_determine_df.arcs.values)
#         }

#         # reset next
#         cars_to_determine_df.loc[~cars_to_determine_df.delivery != True].next = None

#         # True if the car is to deliver 
#         # We assume that the car is to deliver if there is a dealer node in the next candidate set
#         cars_to_deliver = [
#             any(map(lambda i: i not in self.VDC, next_nodes))
#             for vin, next_nodes in NC_dic.items()
#         ]

#         # set delivery
#         cars_to_determine_df.delivery = cars_to_deliver
#         # set next as dealer if delivery car
#         cars_to_determine_df.loc[cars_to_determine_df.delivery, 'next'] = cars_to_determine_df.loc[cars_to_determine_df.delivery, 'dealer']

#         # no delivery cars
#         cars_to_determine_df = cars_to_determine_df[~cars_to_determine_df.delivery]


#         # determine next destination of all cars in the vdc
#         while sum(self.VDC_all_df[vdc].next.isnull()) > 0:

#             NC_df = pd.DataFrame.from_dict(
#             {
#                 vin: [a[1] for a in arcs if a[0]==vdc]
# #                 for (vin,arcs) in zip(cars_to_determine_df.index, cars_to_determine_df.arcs.values)
#                 for (vin,arcs) in zip(self.VDC_all_df[vdc][self.VDC_all_df[vdc].next.isnull()].index, self.VDC_all_df[vdc][self.VDC_all_df[vdc].next.isnull()].arcs.values)
#             },
#             orient='index'
#             )

#             # find the most available next destination
#             most_freq_next_node = pd.concat([NC_df[col].value_counts()*(len(NC_df.columns)-col) for col in NC_df.columns]).groupby(level=0).sum().sort_values(ascending=False).iloc[:1].index[0]

#             # cars that can be sent to the most_freq_next_node
#             vins_to_most_freq_next_node = NC_df[0] == most_freq_next_node
#             if len(NC_df.columns) > 1:
#                 for col in NC_df.columns[1:]:
#                     vins_to_most_freq_next_node |= NC_df[col] == most_freq_next_node


#             self.VDC_all_df[vdc].loc[vins_to_most_freq_next_node.index, 'next'] = most_freq_next_node
    def reset_results_df(self):
        self.result_vdc_to_vdc_df = pd.DataFrame(columns=pd.Index(['depart_vdc', 'depart_time', 'arrival_vdc', 'arrival_time', 'method', 'vins', 'num_vins'], dtype='object'))
        self.result_delivery_df = pd.DataFrame(columns=pd.Index(['vdc', 'shipment_id', 'depart_time', 'path', 'vins', 'cost', 'method'], dtype='object'))
        self.result_route_details_df = pd.DataFrame(columns=pd.Index(['vin', 'loc', 'arrive_time', 'depart_time', 'depart_mode', 'shipment_id', 'shipment_cost'], dtype='object'))


    def get_state(self, link, period):
        
        return g_get_state(self.current_date, period, self.DECISION_INTERVAL, self.MIN_DWELL_TIME, self.NUM_LOOKING_FORWARD, link)

    
    def get_action(self, state):
        if sum(state) == 0: # do nothing...
            return 0
        else:
            state = torch.autograd.Variable(torch.FloatTensor(state).unsqueeze(0))
            q_value = self.model(state)
            action = 1 if q_value[0,0]<q_value[0,1] else 0
    #         # dummy model...
    #         return state[0] >= 7

            return action == 1

    def send_cars(self, link, period, num_to_send):
        
        if num_to_send == 0:
            return

        current_time = (self.current_date) + timedelta(hours=period*self.DECISION_INTERVAL)

        from_vdc, to_vdc = link[0], link[1]

        from_vdc_df = self.VDC_all_df[from_vdc]
        # all cars between two vdcs
        link_all_df = from_vdc_df[(from_vdc_df.next.values == to_vdc)]

        cars_to_send_df = link_all_df[link_all_df.arrival_time <= current_time - timedelta(days=self.MIN_DWELL_TIME)]

        # send cars that stayed longest
        cars_to_send_df = cars_to_send_df.sort_values('arrival_time', axis=0)[:num_to_send]
        
        # data for route details
        vins = list(cars_to_send_df.index)
        loc = [from_vdc] * len(cars_to_send_df)
        prev_arrive_times = list(cars_to_send_df.arrival_time)

        # remove cars to send
        self.VDC_all_df[from_vdc] = from_vdc_df.drop(cars_to_send_df.index)

        # we can send the cars after the time at which the latest car arrived
        time_to_send = cars_to_send_df.arrival_time.max() + timedelta(days=self.MIN_DWELL_TIME)
        
#         print(time_to_send)
#         print(self.current_date)
        
        if time_to_send < self.current_date:
            time_to_send = self.current_date

        link_info = self.Link_info[from_vdc][to_vdc]

        arrive_time = time_to_send + timedelta(hours=link_info['time'])

        depart_time = [time_to_send] * len(cars_to_send_df)
        depart_mode = ['R' if link_info['rail'] else 'T'] * len(cars_to_send_df)
        shipment_cost = [self.calc_transport_cost(link, len(cars_to_send_df))] * len(cars_to_send_df)

        # copy detaframe to manupulation
        cars_to_arrive_df = cars_to_send_df.copy()

        # reset next destinations of cars to send
        cars_to_arrive_df.next = None

        cars_to_arrive_df.vdc = to_vdc
        cars_to_arrive_df.arrival_time = arrive_time
        cars_to_arrive_df.delivery = False

        #                 display(cars_to_arrive_df)

        # append cars to arrive at the destination
        self.VDC_all_df[to_vdc] = pd.concat([self.VDC_all_df[to_vdc], cars_to_arrive_df])   
        
        # update result dataframe
        self.result_vdc_to_vdc_df = self.result_vdc_to_vdc_df.append({
            'depart_vdc': from_vdc, 
            'depart_time': time_to_send, 
            'arrival_vdc': to_vdc,
            'arrival_time': arrive_time, 
            'method': 'rail' if link_info['rail'] else 'truck',
            'vins': list(cars_to_arrive_df.index), # index represents vin...
            'num_vins': len(cars_to_arrive_df)
        }, ignore_index=True)
        
        self.result_route_details_df = self.result_route_details_df.append([{
            'vin': v,
            'loc': l,
            'arrive_time': p,
            'depart_time': dt,
            'depart_mode': dm,
            'shipment_cost': s
        } for v,l,p,dt,dm,s in zip(vins, loc, prev_arrive_times, depart_time, depart_mode, shipment_cost)], ignore_index=True)
        
        
    def calc_transport_cost(self, link, num_cars):
        from_vdc, to_vdc = link[0], link[1]
        link_info = self.Link_info[from_vdc][to_vdc]
        if link_info['rail']:
            return ((2000 + link_info['distance']*3) * (((num_cars-0.1) // 20)+1)) / num_cars # rail
        else:
            return ((200 + link_info['distance']*4) * (((num_cars-0.1) // 10)+1)) / num_cars # truck
        
        
    def process_full_load(self, state, link, period):
        from_vdc, to_vdc = link[0], link[1]
        
        link_info = self.Link_info[from_vdc][to_vdc]

        full_load = 20 if link_info['rail'] else 10

        if state[0] >= full_load:
            num_full_loads = state[0] // full_load
            self.send_cars((from_vdc, to_vdc), period, num_full_loads*full_load)
            return self.get_state(link, period)
        else:
            return state
        
    def process_today_vdc(self, dview=None):
        
        global g_VDC_all_df
        g_VDC_all_df = self.VDC_all_df

        if dview is None:
            for period in range(int(24 / self.DECISION_INTERVAL)):
                for link in self.Links:
                    from_vdc, to_vdc = link[0], link[1]

                    state = self.get_state(link, period)

                    # process full loads
                    state = self.process_full_load(state, link, period)

                    action = self.get_action(state)

                    if action == 1: # send
                        self.send_cars(link, period, state[0])
                    else:
                        self.send_cars_stayed_too_long(link)
        else:
            if dview is not None:
                
                for period in range(int(24 / self.DECISION_INTERVAL)):
                    link_list = self.Links
                    period_list = [period] * len(link_list)
                    current_date_list = [self.current_date] * len(link_list)
                    decision_interval_list = [self.DECISION_INTERVAL] * len(link_list)
                    min_dwelltime_list = [self.MIN_DWELL_TIME] * len(link_list)
                    num_looking_forward_list = [self.NUM_LOOKING_FORWARD] * len(link_list)

                    dview['g_VDC_all_df'] =  self.VDC_all_df
                    
                    state_list = dview.map_sync(g_get_state, current_date_list, period_list, decision_interval_list, min_dwelltime_list, num_looking_forward_list, link_list)

                    for state, link in zip(state_list, link_list):
                        # process full loads
                        state = self.process_full_load(state, link, period)

                        action = self.get_action(state)

                        if action == 1: # send
                            self.send_cars(link, period, state[0])
                        else:
                            self.send_cars_stayed_too_long(link)

        
    
    def send_cars_stayed_too_long(self, link):
        from_vdc, to_vdc = link[0], link[1]
        from_vdc_df = self.VDC_all_df[from_vdc]
        if len(from_vdc_df) > 0:
            # all cars between two vdcs
            link_all_df = from_vdc_df[(from_vdc_df.next.values == to_vdc)]

            if len(link_all_df) > 0:
                cars_to_send_df = link_all_df[link_all_df.arrival_time <= self.current_date - timedelta(days=self.MAX_STAY_DAYS)]

                if len(cars_to_send_df) > 0:
                    self.send_cars(link, 0, 20)
                
                

    def update_inventory_status(self):
        inv_dic = {
            v : [
                sum((self.VDC_all_df[v].delivery==False) & (self.VDC_all_df[v].arrival_time<=self.current_date)), 
                sum((self.VDC_all_df[v].delivery==True) & (self.VDC_all_df[v].arrival_time<=self.current_date)), 
                sum(self.VDC_all_df[v].arrival_time<=self.current_date)
            ]
            for v in self.VDC
        }

        inv_df = pd.DataFrame.from_dict(inv_dic, orient='index', columns=['To VDC', 'To dealer', 'Total']).transpose()
        self.inventory_df = inv_df 
        return inv_df
        
    def solve_delivery(self, vdc, drop_delivered_cars=True):
        cars_to_deliver_df = self.VDC_all_df[vdc][self.VDC_all_df[vdc].delivery == True]
        # cars_to_deliver_df.index.name = 'vin'

        vrp_cost, _, adjusted_vrp_sol,_,_ = self.vrp.solve_vrp(
            vdc, 
            str(self.current_date)[:10], 
            cars_to_deliver_df, 
            self.VRP_PERIODS, 
            AV_dist=300,
            AV=self.vehicle_type==2,
            log=False,
            time_out=self.VRP_TIMEOUT
        )


        delivered_cars = [
            vin
            for shipment in adjusted_vrp_sol
            for vin,_,_ in adjusted_vrp_sol[shipment]['vins']
        ]

        # correct depart time....
        for s_id,route in adjusted_vrp_sol.items():
            depart_time = route['path'][0][1]

            vins = [vin[0] for vin in route['vins']]
            real_depart_time = cars_to_deliver_df.loc[vins].arrival_time.max()

            if real_depart_time > depart_time:
                shift_time = real_depart_time - depart_time

                route['path'] = [
                    (i,t+shift_time,l) for (i,t,l) in route['path']
                ]
                route['vins'] = [
                    (v,t+shift_time,l) for (v,t,l) in route['vins']
                ]

        # update result
        self.update_result_for_delivery(vdc, adjusted_vrp_sol)

        if drop_delivered_cars:
            # drop delivered cars
            self.VDC_all_df[vdc] = self.VDC_all_df[vdc][~self.VDC_all_df[vdc].vin.isin(delivered_cars)]



        return delivered_cars
    
        
    
    def solve_delivery_parallel(self, dview, drop_delivered_cars=True):    
        all_cars_to_deliver_df = [
            self.VDC_all_df[vdc][self.VDC_all_df[vdc].delivery == True]
            for vdc in self.VDC
        ]
                

        all_vrp_solutions_list = dview.map_sync(
            par_solve_vrp, 
            self.VDC, 
            [str(self.current_date)[:10]] * len(self.VDC), 
            all_cars_to_deliver_df, 
            [self.VRP_PERIODS] * len(self.VDC), 
            [300] * len(self.VDC), 
            [self.vehicle_type==2] * len(self.VDC),
            [False] * len(self.VDC),
            [self.VRP_TIMEOUT] * len(self.VDC)
        )

        all_delivered_cars = {
            vdc: [
                vin
                for shipment in adjusted_vrp_sol
                for vin,_,_ in adjusted_vrp_sol[shipment]['vins']
            ]   
            for (vdc, (vrp_cost, _, adjusted_vrp_sol,_,_)) in zip(self.VDC, all_vrp_solutions_list)
        }

        
#         print(all_vrp_solutions_list[0][1]['2015/01/04-3A-7-1-99'])
        
        # correct depart time....
        for (vrp_cost, _, adjusted_vrp_sol,_,_), cars_df in zip(all_vrp_solutions_list, all_cars_to_deliver_df):
            for s_id,route in adjusted_vrp_sol.items():
                depart_time = route['path'][0][1]

                vins = [vin[0] for vin in route['vins']]
                real_depart_time = cars_df.loc[vins].arrival_time.max()

                if real_depart_time > depart_time:
                    shift_time = real_depart_time - depart_time

                    route['path'] = [
                        (i,t+shift_time,l) for (i,t,l) in route['path']
                    ]
                    route['vins'] = [
                        (v,t+shift_time,l) for (v,t,l) in route['vins']
                    ]
#         print(all_vrp_solutions_list[0][1]['2015/01/04-3A-7-1-99'])


        # update result
        for (vdc, (vrp_cost, _, adjusted_vrp_sol,_,_)) in zip(self.VDC, all_vrp_solutions_list):
            self.update_result_for_delivery(vdc, adjusted_vrp_sol)

        if drop_delivered_cars:
            # drop delivered cars
            for vdc in self.VDC:
                self.VDC_all_df[vdc] = self.VDC_all_df[vdc][~self.VDC_all_df[vdc].vin.isin(all_delivered_cars[vdc])]

        return {v:len(all_delivered_cars[v]) for v in self.VDC}

    def update_result_for_delivery(self, vdc, adjusted_vrp_sol):
        if len(adjusted_vrp_sol) > 0:
            
            self.result_delivery_df = self.result_delivery_df.append([{
                'vdc': vdc,
                'shipment_id': shipment,
                'depart_time': sol['path'][0][1],
                'path': sol['path'],
                'vins': sol['vins'],
                'cost': sol['cost'],
                'method': 'truck',
            } for shipment, sol in adjusted_vrp_sol.items()], ignore_index=True)
        
        for shipment, sol in adjusted_vrp_sol.items():
            
            mode = 'T' if shipment.find('AV') == -1 else 'A'
            
            vins = [v[0] for v in sol['vins']]
#             loc = [vdc] * len(vins)
            loc = [sol['path'][0][0]] * len(vins)
            prev_arrive_times = list(self.VDC_all_df[vdc].loc[vins].arrival_time)
            depart_time = [sol['path'][0][1]] * len(vins)
            depart_mode = ['T'] * len(vins)
            shipment_id = [shipment] * len(vins)
            

            self.result_route_details_df = self.result_route_details_df.append([{
                'vin': v,
                'loc': l,
                'arrive_time': p,
                'depart_time': dt,
                'depart_mode': dm,
                'shipment_id': s
            } for v,l,p,dt,dm,s in zip(vins, loc, prev_arrive_times, depart_time, depart_mode, shipment_id)], ignore_index=True)
            
#             if sum(self.result_route_details_df.arrive_time > self.result_route_details_df.depart_time)>0:
#                 print(prev_arrive_times)
#                 print(depart_time)

            vins = [v[0] for v in sol['vins']]
            loc = list(self.VDC_all_df[vdc].loc[vins].dealer)
            arrive_times = [v[1] for v in sol['vins']]
            depart_time = arrive_times
            depart_mode = ['T'] * len(vins)

            self.result_route_details_df = self.result_route_details_df.append([{
                'vin': v,
                'loc': l,
                'arrive_time': p,
                'depart_time': dt,
                'depart_mode': dm
#                 'shipment_id': s
            } for v,l,p,dt,dm,s in zip(vins, loc, arrive_times, depart_time, depart_mode, shipment_id)], ignore_index=True)
            
#             if shipment=='2015/01/04-3A-7-1-99':
#                 qqq = 1/0
    
    
    def save_today_snapshot(self, savedir='res'):
        today_snapshot = {
            'date': self.current_date,
            'vdc_status': self.VDC_all_df,
            'vdc_to_vdc': self.result_vdc_to_vdc_df,
            'delivery': self.result_delivery_df,
            'route_details_today': self.result_route_details_df
        }

        pickle.dump(today_snapshot, open(f'{savedir}/{self.current_date.strftime("%Y-%m-%d")}.dump', 'wb'))


    def solve(self, from_date=None, to_date=None, dview=None, save_dir='res'):
        if from_date is not None:
            self.current_date = pd.to_datetime(from_date)

        if to_date is not None:
            end_date = pd.to_datetime(to_date)
        else:
            end_date = pd.to_datetime(f'{self.year}-12-31')

        start_solve = timer()

        save_everyday_status = True
                                         
        num_total_ordered = 0
        num_total_delivered = 0

        while self.current_date <= end_date:

            self.reset_results_df()

            num_total_ordered += self.append_cars_arrive_today()

            print(f'{self.current_date.strftime("%Y/%m/%d")} Solving time: {timer()  - start_solve:0.2f} seconds. Total ordered: {num_total_ordered}. Total delivered: {num_total_delivered}')

            self.determine_next_dest()

            print(f'\tProcess VDC-VDC decisions{(" (parallel:use "+str(len(dview))+" cores)") if dview is not None else ""}', end='')
            start_vdc = timer()
            self.process_today_vdc(dview)
            print(f': {timer()  - start_vdc:0.2f} seconds.')

            self.determine_next_dest()

            start_vrp = timer()
            if dview is not None:
                print(f'\tSolve delivery problems (parallel:use {len(dview)} cores): ', end='')
                self.vrp = VRP(list(self.VDC))
                dview['vrp'] = self.vrp
                dview.execute('vrp.location = g_location;vrp.dist = g_dist;vrp.ind = g_ind')
                all_deliverted_cars = self.solve_delivery_parallel(dview)
            else:
                print(f'\tSolve delivery problems: ', end='')
                all_deliverted_cars ={}
                for v in self.VDC:
                    print(f'{v}', end=' ')
                    delivered_cars = self.solve_delivery(v)
                    all_deliverted_cars[v] = len(delivered_cars)
            print(f': {timer()  - start_vrp:0.2f} seconds.')

            self.current_date += timedelta(days=1)

            self.update_inventory_status()
            self.inventory_df.loc['Delivered today'] = (all_deliverted_cars)

            print('\tInventory status:')
            display(self.inventory_df)

            num_total_delivered += sum(all_deliverted_cars.values())
            

            if save_everyday_status:
                self.save_today_snapshot(savedir=save_dir)


# helper functions for parallel
def par_solve_vrp(vdc, date, cars_to_deliver_df, period, log=True, AV_dist=200, time_out=10, AV=False):
    return vrp.solve_vrp(vdc, date, cars_to_deliver_df, period, log, AV_dist, time_out, AV)

def g_get_state(current_date, period, decision_interval, min_dwell_time, num_looking_forward, link):

    current_time = (current_date) + timedelta(hours=period*decision_interval)

    from_vdc, to_vdc = link[0], link[1]

    state = []
                                    
    from_vdc_df = g_VDC_all_df[from_vdc]
    to_vdc_df = g_VDC_all_df[to_vdc]

    if len(from_vdc_df) > 0:

        # all cars between two vdcs
        link_all_df = from_vdc_df[(from_vdc_df.next.values == to_vdc)]

        if len(link_all_df) > 0:
            # numbers of cars that can be transported at t, t+1, ..., t+NUM_LOOKING_FORWARD
            link_num_cars = [
                sum(link_all_df.arrival_time <= current_time - timedelta(days=min_dwell_time) + timedelta(hours=t*decision_interval))
                for t in range(num_looking_forward)
            ]
        else:
            link_num_cars = [0] * (num_looking_forward)

        state.extend(link_num_cars)

        if link_num_cars[0] > 0 and len(to_vdc_df) > 0:
            # numbers of cars of destination vdc at t, t+1, ..., t+NUM_LOOKING_FORWARD
            inv_cars = [
                sum(to_vdc_df.arrival_time <= current_time - timedelta(days=min_dwell_time) + timedelta(hours=t*decision_interval))
                for t in range(num_looking_forward)
            ]
        else:
            inv_cars = [0] * (num_looking_forward)

        state.extend(inv_cars)

    else:
        state = [0] * (num_looking_forward) * 2

    # construct a state
    state = np.array(state)

    return state

In [3]:

import ipyparallel as ipp
rc = ipp.Client()
dview = rc[:]

expdir = '../../ExpNew'

distance_matrix = pickle.load(open(f'{expdir}/dist_mat_9361.dump', 'rb'))
location_index_dic = pickle.load(open(f'{expdir}/location_index_9361.dump', 'rb'))        
location = pd.read_excel(f'{expdir}/Input_Cost%2C+Location.xlsx')



# initialize parallism                                         
def init_parallel():
    if dview is not None:
        with dview.sync_imports():
            from ortools.constraint_solver import pywrapcp
            from ortools.constraint_solver import routing_enums_pb2
            from datetime import date
            import pickle
            import pandas as pd
            %px pd = pandas
            import numpy as np
            %px np = numpy
            from datetime import datetime, timedelta
#             from solver import *
#             from vrp import *
        # push local data and functions to remotes
#         dview.execute('from solver import *')
        dview['VRP'] = VRP
        dview['g_location'] = location
        dview['g_dist'] = distance_matrix
        dview['g_ind'] = location_index_dic
#         dview['vrp'] = solver.vrp
        dview['par_solve_vrp'] = par_solve_vrp
        dview['g_get_state'] = g_get_state
        


init_parallel()
dview

importing pywrapcp from ortools.constraint_solver on engine(s)
importing routing_enums_pb2 from ortools.constraint_solver on engine(s)
importing date from datetime on engine(s)
importing pickle on engine(s)
importing pandas on engine(s)
importing InteractiveShell from IPython.core.interactiveshell on engine(s)
importing numpy on engine(s)
importing datetime,timedelta from datetime on engine(s)


<DirectView [0, 1, 2, 3]>

In [4]:
# 2016

expdir = '../../ExpNew'
snapshot_dir = '../../ExpNew/2016/s4'
save_dir = '../../ExpNew/2016/s4'

start_date = '2016-12-27'
end_date = '2017-1-31'

year = '2016'

reset_interval = 1000

# S1
# route_type = 1
# vehicle_type = 1

# S2
# route_type = 2
# vehicle_type = 1

# S3
# route_type = 1
# vehicle_type = 2

# S4
route_type = 2
vehicle_type = 2

cur_time = pd.to_datetime(start_date)


while cur_time <= pd.to_datetime(end_date):
    solver = INFORMSsolver(distance_matrix, location_index_dic, location, expdir=expdir, year=year, route_type=route_type, vehicle_type=vehicle_type, restore_snapshot=f"{snapshot_dir}/{cur_time.strftime('%Y-%m-%d.dump')}")

    batch_end_time = pd.to_datetime(end_date) if pd.to_datetime(end_date) < cur_time + timedelta(days=reset_interval) else cur_time + timedelta(days=reset_interval)
    
    solver.solve(to_date=batch_end_time.strftime('%Y-%m-%d'), dview=dview, save_dir=save_dir)
                           
    cur_time += timedelta(days=reset_interval+1)



2016/12/27 Solving time: 6.64 seconds. Total ordered: 1559. Total delivered: 0
	0 route changes!
	Process VDC-VDC decisions (parallel:use 4 cores)

Y:\anaconda\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


: 14.62 seconds.
	0 route changes!
	Solve delivery problems (parallel:use 4 cores): : 16.55 seconds.
	Inventory status:


,3A,3F,4J,7J,7M,BC,BE,BM,CE,CW,DI,DO,DV,DW,DZ,EC,FF,GU,JC,LM,MN,MR,NM,NZ,OX,PB,PH,QT,QW,RJ,RO,RS,RX,SO,SU,SZ,UL,VE,VG,VH,VW,WH,WK,WL
To VDC,1284,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,594,0,0,0,0,0,0,0,0,0,0,0,0,0,1289,0,0,2184,0,0,0,0,0,0,0,0,0,0
To dealer,5,0,7,37,0,11,0,0,11,0,6,32,0,0,6,11,0,14,38,0,8,0,11,0,6,0,0,17,0,0,0,23,0,0,8,1,0,0,43,16,0,0,0,0
Total,1289,0,7,37,0,11,0,0,11,0,6,32,0,0,6,11,594,14,38,0,8,0,11,0,6,0,0,17,0,0,1289,23,0,2184,8,1,0,0,43,16,0,0,0,0
Delivered today,80,30,118,51,6,89,79,0,77,33,99,116,0,0,59,62,18,95,132,120,167,0,57,0,213,20,35,75,0,0,9,76,0,83,118,113,40,20,9,39,20,0,0,0


2016/12/28 Solving time: 46.70 seconds. Total ordered: 2934. Total delivered: 2358
	12 route changes!
	Process VDC-VDC decisions (parallel:use 4 cores): 6.84 seconds.
	0 route changes!
	Solve delivery problems (parallel:use 4 cores): : 16.90 seconds.
	Inventory status:


...

,3A,3F,4J,7J,7M,BC,BE,BM,CE,CW,DI,DO,DV,DW,DZ,EC,FF,GU,JC,LM,MN,MR,NM,NZ,OX,PB,PH,QT,QW,RJ,RO,RS,RX,SO,SU,SZ,UL,VE,VG,VH,VW,WH,WK,WL
To VDC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
To dealer,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Delivered today,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


2017/01/30 Solving time: 390.34 seconds. Total ordered: 5860. Total delivered: 24481
	0 route changes!
	Process VDC-VDC decisions (parallel:use 4 cores): 0.18 seconds.
	0 route changes!
	Solve delivery problems (parallel:use 4 cores): : 0.35 seconds.
	Inventory status:


,3A,3F,4J,7J,7M,BC,BE,BM,CE,CW,DI,DO,DV,DW,DZ,EC,FF,GU,JC,LM,MN,MR,NM,NZ,OX,PB,PH,QT,QW,RJ,RO,RS,RX,SO,SU,SZ,UL,VE,VG,VH,VW,WH,WK,WL
To VDC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
To dealer,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Delivered today,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


2017/01/31 Solving time: 391.29 seconds. Total ordered: 5860. Total delivered: 24481
	0 route changes!
	Process VDC-VDC decisions (parallel:use 4 cores): 0.21 seconds.
	0 route changes!
	Solve delivery problems (parallel:use 4 cores): : 0.34 seconds.
	Inventory status:


,3A,3F,4J,7J,7M,BC,BE,BM,CE,CW,DI,DO,DV,DW,DZ,EC,FF,GU,JC,LM,MN,MR,NM,NZ,OX,PB,PH,QT,QW,RJ,RO,RS,RX,SO,SU,SZ,UL,VE,VG,VH,VW,WH,WK,WL
To VDC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
To dealer,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Delivered today,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 2015

expdir = '../../ExpNew'
snapshot_dir = '../../ExpNew/2015/s4'
save_dir = '../../ExpNew/2015/s4'

start_date = '2015-12-24'
end_date = '2015-12-31'

year = '2015'

reset_interval = 1000

# S1
# route_type = 1
# vehicle_type = 1

# S2
# route_type = 2
# vehicle_type = 1

# S3
# route_type = 1
# vehicle_type = 2

# S4
route_type = 2
vehicle_type = 2

cur_time = pd.to_datetime(start_date)


while cur_time <= pd.to_datetime(end_date):
    solver = INFORMSsolver(distance_matrix, location_index_dic, location, expdir=expdir, year=year, route_type=route_type, vehicle_type=vehicle_type, restore_snapshot=f"{snapshot_dir}/{cur_time.strftime('%Y-%m-%d.dump')}")

    batch_end_time = pd.to_datetime(end_date) if pd.to_datetime(end_date) < cur_time + timedelta(days=reset_interval) else cur_time + timedelta(days=reset_interval)
    
    solver.solve(to_date=batch_end_time.strftime('%Y-%m-%d'), dview=dview, save_dir=save_dir)
                           
    cur_time += timedelta(days=reset_interval+1)



2015/12/24 Solving time: 10.38 seconds. Total ordered: 2401. Total delivered: 0
	12 route changes!
	Process VDC-VDC decisions (parallel:use 4 cores)

Y:\anaconda\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


: 13.31 seconds.
	0 route changes!
	Solve delivery problems (parallel:use 4 cores): : 18.61 seconds.
	Inventory status:


,3A,3F,4J,7J,7M,BC,BE,BM,CE,CW,DI,DO,DV,DW,DZ,EC,FF,GU,JC,LM,MN,MR,NM,NZ,OX,PB,PH,QT,QW,RJ,RO,RS,RX,SO,SU,SZ,UL,VE,VG,VH,VW,WH,WK,WL
To VDC,1496,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1440,0,0,0,0,0,0,0,0,0,0,0,0,0,1754,0,0,2447,0,0,0,0,0,0,0,0,0,0
To dealer,17,1,10,19,0,16,7,0,16,0,5,9,0,0,20,31,0,29,0,0,41,0,7,0,24,0,0,7,0,0,0,0,0,0,2,7,4,2,7,8,0,0,0,0
Total,1513,1,10,19,0,16,7,0,16,0,5,9,0,0,20,31,1440,29,0,0,41,0,7,0,24,0,0,7,0,0,1754,0,0,2447,2,7,4,2,7,8,0,0,0,0
Delivered today,69,214,92,62,4,109,77,0,158,2,55,185,69,16,71,76,19,45,48,80,137,0,58,0,123,0,20,119,0,6,21,95,10,87,65,80,79,49,18,23,0,0,0,40


2015/12/25 Solving time: 54.62 seconds. Total ordered: 4511. Total delivered: 2481
	9 route changes!
	Process VDC-VDC decisions (parallel:use 4 cores): 8.21 seconds.
	1 route changes!
	Solve delivery problems (parallel:use 4 cores): : 15.60 seconds.
	Inventory status:


...

2015/12/29 Solving time: 208.75 seconds. Total ordered: 10037. Total delivered: 12377
	20 route changes!
	Process VDC-VDC decisions (parallel:use 4 cores): 6.73 seconds.
	1 route changes!
	Solve delivery problems (parallel:use 4 cores): : 18.23 seconds.
	Inventory status:


,3A,3F,4J,7J,7M,BC,BE,BM,CE,CW,DI,DO,DV,DW,DZ,EC,FF,GU,JC,LM,MN,MR,NM,NZ,OX,PB,PH,QT,QW,RJ,RO,RS,RX,SO,SU,SZ,UL,VE,VG,VH,VW,WH,WK,WL
To VDC,700,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,647,0,0,0,0,0,0,0,0,0,0,0,0,0,786,0,0,1062,0,0,0,0,0,0,0,0,0,0
To dealer,16,5,41,0,0,38,0,8,0,0,14,21,0,0,11,22,0,14,15,0,11,8,4,0,0,0,0,19,0,0,0,18,0,20,21,2,0,0,27,6,0,0,0,0
Total,716,5,41,0,0,38,0,8,0,0,14,21,0,0,11,22,647,14,15,0,11,8,4,0,0,0,0,19,0,0,786,18,0,1082,21,2,0,0,27,6,0,0,0,0
Delivered today,94,186,183,40,0,125,97,59,20,0,60,87,20,20,175,56,12,15,99,120,125,32,39,12,191,0,60,69,0,0,5,51,10,57,101,73,40,109,19,8,0,0,0,0


2015/12/30 Solving time: 241.72 seconds. Total ordered: 10477. Total delivered: 14846
	17 route changes!
	Process VDC-VDC decisions (parallel:use 4 cores): 6.57 seconds.
	0 route changes!
	Solve delivery problems (parallel:use 4 cores): : 17.17 seconds.
	Inventory status:


,3A,3F,4J,7J,7M,BC,BE,BM,CE,CW,DI,DO,DV,DW,DZ,EC,FF,GU,JC,LM,MN,MR,NM,NZ,OX,PB,PH,QT,QW,RJ,RO,RS,RX,SO,SU,SZ,UL,VE,VG,VH,VW,WH,WK,WL
To VDC,545,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,577,0,0,0,0,0,0,0,0,0,0,0,0,0,676,0,0,782,0,0,0,0,0,0,0,0,0,0
To dealer,13,32,0,0,0,36,2,10,2,0,9,2,0,6,11,6,1,29,15,0,15,21,0,0,16,0,0,21,0,0,0,39,0,0,16,3,0,2,39,13,0,0,0,0
Total,558,32,0,0,0,36,2,10,2,0,9,2,0,6,11,6,578,29,15,0,15,21,0,0,16,0,0,21,0,0,676,39,0,782,16,3,0,2,39,13,0,0,0,0
Delivered today,83,71,185,20,5,143,77,18,58,24,124,164,14,22,141,64,1,85,132,40,105,47,64,0,50,60,0,59,0,0,9,60,10,57,92,61,60,37,30,13,0,0,0,18


2015/12/31 Solving time: 272.55 seconds. Total ordered: 10789. Total delivered: 17149
	23 route changes!
	Process VDC-VDC decisions (parallel:use 4 cores): 6.30 seconds.
	0 route changes!
	Solve delivery problems (parallel:use 4 cores): 